#                               Batsman Score prediction for each match

In [172]:
####Visualizations
#### Importing dependencies 
import numpy as np
import pandas as pa
import seaborn as se
import matplotlib.pyplot as plt
from IPython.display import HTML

In [173]:
ball_by_ball = pa.read_csv("./train/train/ball_by_ball_data.csv")
match_data = pa.read_csv("./train/train/match_data.csv")
player_roster = pa.read_csv("./train/train/player_rosters.csv")
key_teams = pa.read_csv("./train/train/key_teams.csv")
matchs_2018=pa.read_csv("./test/test/matches_2018.csv")
player_prediction_2018=pa.read_csv("./test/test/player_predictions.csv")

### Lets see the players of 2018.

In [174]:
### Players of 2018.

players_2018=player_roster[player_roster['Season']==2018][['player_id','Player','Team']].reset_index()
del players_2018['index']
print(players_2018.head(10))

### Teams in 2018.

teams_2018=list(players_2018['Team'].unique())

print(teams_2018)


   player_id                Player Team
0          1  Mahendra Singh Dhoni  CSK
1          3   Ravichandran Ashwin  KXI
2         12         Parthiv Patel  RCB
3         24          Suresh Raina  CSK
4         30          Rohit Sharma   MI
5         47        AB De Villiers  RCB
6         52           Amit Mishra   DD
7         55          Manoj Tiwary  KXI
8         57        Shikhar Dhawan  SRH
9         58        Gautam Gambhir   DD
['CSK', 'KXI', 'RCB', 'MI', 'DD', 'SRH', 'KKR', 'RR']


## Filtering only players who are playing in 2018. 

Because i dont want to predict scores of the players who are not playing in 2018. So i keep only 2018 players in my 2008 to 2017 data.

In [175]:
#### Filter only the 2018 players.

ball_by_ball_only_2018=ball_by_ball[ball_by_ball['batsman_id'].isin(list(players_2018['player_id']))]

In [176]:
ball_by_ball_only_2018=ball_by_ball_only_2018.reset_index()
del ball_by_ball_only_2018['index']

In [177]:
### Find the frquency of the teams from 2008 to 2017.

freq_of_teams=sum([list(match_data['team1_id']),list(match_data['team2_id'])],[])

In [178]:
(pa.Series(freq_of_teams)).value_counts()

MI     157
RCB    152
KKR    148
KXI    148
DD     147
CSK    131
RR     118
SRH     76
DC      75
PW      46
RPS     30
GL      30
KTK     14
dtype: int64

## Changing DC to SRH

In [179]:
### Ownership being transferred from Deccan Chronicle Holdings Ltd to Sun TV channel Network in 2013.
### So Renaming the DC team to SRH is good option. Since they are the same teams.

teams_renaming = {'DC':'SRH'}

match_data1=match_data.replace({'team1_id':teams_renaming,'team2_id':teams_renaming,'toss_winner':teams_renaming,'winner':teams_renaming})
ball_by_ball_only_2018=ball_by_ball_only_2018.replace({'batting_team':teams_renaming,'bowling_team':teams_renaming})
ball_by_ball_only_2018=ball_by_ball_only_2018.merge(match_data[['match_id','season','venue_id']], on='match_id')

In [180]:
ball_by_ball_only_2018.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,batsman_id,non_striker,non_striker_id,bowler,bowler_id,batsman_runs,extra_runs,player_dismissed,player_dismissed_id,dismissal_kind,is_super_over,season,venue_id
0,2017_1,1,SRH,RCB,1,1,David Warner,207,Shikhar Dhawan,57,Tymal Mills,621,0,0,NaN,NaN,NaN,0,2017,4
1,2017_1,1,SRH,RCB,1,2,David Warner,207,Shikhar Dhawan,57,Tymal Mills,621,0,0,NaN,NaN,NaN,0,2017,4
2,2017_1,1,SRH,RCB,1,3,David Warner,207,Shikhar Dhawan,57,Tymal Mills,621,4,0,NaN,NaN,NaN,0,2017,4
3,2017_1,1,SRH,RCB,1,4,David Warner,207,Shikhar Dhawan,57,Tymal Mills,621,0,0,NaN,NaN,NaN,0,2017,4
4,2017_1,1,SRH,RCB,1,5,David Warner,207,Shikhar Dhawan,57,Tymal Mills,621,0,2,NaN,NaN,NaN,0,2017,4


In [181]:
ball_by_ball_only_2018['seasonal_match_id']=[row.split('_')[1] for row in ball_by_ball_only_2018['match_id']]

## Seperating 1st and 2nd innings

Since we are predicting the scores for both 1st innings and 2nd innings seperately, i split the data into two innings. Here i didnt take the 3rd and 4th innings since it comes under the superover concept.

In [182]:
ball_by_ball_only_2018['season']=ball_by_ball_only_2018['season'].apply(int)
ball_by_ball_only_2018['seasonal_match_id']=ball_by_ball_only_2018['seasonal_match_id'].apply(int)

In [183]:
#### Seperating 1st and 2nd innings.

ball_by_ball_only_2018_1stinn=ball_by_ball_only_2018[ball_by_ball_only_2018['inning']==1]
ball_by_ball_only_2018_2ndinn=ball_by_ball_only_2018[ball_by_ball_only_2018['inning']==2]

# 1st Innings

## Calculating Strike rate, Average, matches played

Let me explain here, Strike rate, average, matches played i calculated cummulatively. So for each match i put the over all strike rate from all previous matches. Here what i mean is im taking the predictor variables as the strike rate, average, matches played from all the previous matches and predict the present match player score.

In [184]:
score_card_1st_inn=ball_by_ball_only_2018_1stinn.groupby(['match_id','seasonal_match_id','season','venue_id','batting_team','bowling_team','batsman','batsman_id'], as_index=False)['batsman_runs'].agg({'ball_palyed':'count','batsman_runs':'sum'})

In [185]:
score_card_1st_inn['unique_id']=(score_card_1st_inn['match_id']).apply(str)+"_"+(score_card_1st_inn['batsman_id']).apply(str)

In [186]:
score_card_1st_inn=score_card_1st_inn.sort_values(['batsman_id','season','seasonal_match_id'])

In [187]:
#### Adding up the total Runs scored for each match Cummulatively
score_card_1st_inn['cumm_runs']=score_card_1st_inn.groupby(['batsman_id'])['batsman_runs'].cumsum()
#### Adding the total balls faced for the each match
score_card_1st_inn['B/F']=score_card_1st_inn.groupby(['batsman_id'])['ball_palyed'].cumsum()

In [188]:
#### Calculating the strike rate formula: total_no_of_runs/total_balls_faced


score_card_1st_inn['strike_rate']=np.round((score_card_1st_inn['cumm_runs']/score_card_1st_inn['B/F'])*100,2)



In [189]:
#### To calculate the Average the of the batsman for each match.

#### This is number of matches played.
score_card_1st_inn['matches_played']=(score_card_1st_inn.groupby(['batsman_id'])['B/F'].rank(ascending=True)).apply(int)


#### Average of batsman formula: Runs_scored/Matches_palyed.

score_card_1st_inn['Averge_of_batsman']=np.round((score_card_1st_inn['cumm_runs']/score_card_1st_inn['matches_played']),2)




### In 2018 for CSK few match venues got changed from chennai stadium to pune. 

In [190]:
matchs_2018

#### change the venue id:
#### Due to cauvery issue in chennai. The stadium change from chennai to maharastra cricket association stadium with id 32
#### Match ids to change:   
""" 2018_17
    2018_27
    2018_30
    2018_35
    2018_46
    2018_56 """


ids_to_change=['2018_17',
'2018_27',
'2018_30',
'2018_35',
'2018_46',
'2018_56']


for i in list(matchs_2018[matchs_2018['match_id'].isin(ids_to_change)].index):
    matchs_2018.iloc[i,4]='Maharashtra Cricket Association Stadium'
    matchs_2018.iloc[i,5]=32



In [191]:
match_2018_details=player_prediction_2018[['unique_id','match_id','player_id','team_id']].merge(matchs_2018[['match_id','team1_id','team2_id','venue_id']], on='match_id')

In [192]:
match_2018_details['batting_team']='nan'
match_2018_details['bowling_team']='nan'

In [193]:
for i in range(len(match_2018_details)):
    if match_2018_details.iloc[i,3]==match_2018_details.iloc[i,4]:
        match_2018_details.iloc[i,7]=match_2018_details.iloc[i,3]
        match_2018_details.iloc[i,8]=match_2018_details.iloc[i,5]
    elif match_2018_details.iloc[i,3]==match_2018_details.iloc[i,5]:
        match_2018_details.iloc[i,7]=match_2018_details.iloc[i,5]
        match_2018_details.iloc[i,8]=match_2018_details.iloc[i,4]
    

In [194]:
del match_2018_details['team_id']
del match_2018_details['team1_id']
del match_2018_details['team2_id']

In [195]:
match_2018_details.head()

,unique_id,match_id,player_id,venue_id,batting_team,bowling_team
0,2018_1_1,2018_1,1,1,CSK,MI
1,2018_1_24,2018_1,24,1,CSK,MI
2,2018_1_114,2018_1,114,1,CSK,MI
3,2018_1_131,2018_1,131,1,CSK,MI
4,2018_1_175,2018_1,175,1,CSK,MI


In [196]:
#### Who are the player not there, means not played even a single ball in past matchs 2008 to 2017 let's dont consider in prediction

np.sort(match_2018_details['player_id'].unique())


array([  1,   3,  12,  24,  30,  47,  52,  55,  57,  58,  59,  63,  71,
        79,  82,  84,  88, 102, 114, 118, 120, 122, 126, 131, 141, 149,
       153, 157, 171, 175, 177, 189, 207, 208, 211, 249, 250, 254, 271,
       305, 309, 311, 315, 321, 322, 333, 337, 340, 341, 343, 351, 359,
       361, 373, 378, 386, 402, 411, 412, 429, 430, 433, 437, 439, 443,
       449, 456, 458, 464, 465, 467, 469, 471, 478, 479, 480, 484, 485,
       487, 500, 502, 507, 511, 512, 514, 518, 521, 526, 528, 529, 531,
       535, 537, 540, 548, 551, 553, 559, 562, 564, 565, 567, 570, 571,
       572, 573, 574, 581, 582, 590, 593, 594, 596, 598, 600, 606, 608,
       610, 613, 614, 617, 618, 619, 622, 625, 626, 630, 631, 632, 633,
       634, 635, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647,
       648, 649, 650, 651, 652, 654, 655, 656, 657, 658, 659, 661, 662,
       663, 664, 665, 666, 668, 669, 670, 671, 672, 673, 674, 675, 676,
       677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 68

In [197]:
np.sort(ball_by_ball_only_2018['batsman_id'].unique())

array([  1,   3,  12,  24,  30,  47,  52,  55,  57,  58,  59,  63,  71,
        79,  82,  84,  88, 102, 114, 118, 120, 122, 126, 131, 141, 149,
       153, 157, 171, 175, 177, 189, 207, 208, 211, 249, 250, 254, 271,
       305, 309, 311, 315, 321, 322, 333, 337, 340, 341, 343, 351, 359,
       361, 373, 378, 386, 402, 411, 412, 429, 430, 433, 437, 439, 443,
       456, 458, 465, 467, 469, 471, 478, 479, 480, 484, 485, 487, 500,
       502, 507, 511, 512, 514, 518, 521, 526, 528, 529, 535, 537, 540,
       548, 551, 553, 559, 562, 564, 565, 567, 570, 572, 573, 574, 581,
       582, 590, 594, 598, 600, 606, 610, 625, 626, 630, 631, 632, 633,
       634, 647, 649, 655, 661, 664, 668, 677, 682, 691], dtype=int64)

### Non predictable players. They might be new in 2018 season or in previous matches being in the team they didnt even play a single ball

In [198]:
### These are the players not played even single ball from 2008 to 2017. So my view is we cannot predicit the player with out having
### any previous(past) data.
not_predictable_players=[player for player in np.sort(match_2018_details['player_id'].unique()) if player not in np.sort(ball_by_ball_only_2018['batsman_id'].unique()) ]

In [199]:
not_predictable_players

[449,
 464,
 531,
 571,
 593,
 596,
 608,
 613,
 614,
 617,
 618,
 619,
 622,
 635,
 637,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 648,
 650,
 651,
 652,
 654,
 656,
 657,
 658,
 659,
 662,
 663,
 665,
 666,
 669,
 670,
 671,
 672,
 673,
 674,
 675,
 676,
 678,
 679,
 680,
 681,
 683,
 684,
 685,
 686,
 687,
 688,
 689,
 690,
 692,
 693,
 696]

## Test set for 2018 matches (1st innings)

What i did is took the strike rate,average and matches played from 2008 to very last match the player played before 2018. So that based on his overall performance in IPL career we predict his each match score in 2018.

In [200]:
test_data_prep_1st_inn=score_card_1st_inn[['batsman_id','batsman','matches_played','strike_rate','Averge_of_batsman']].sort_values(['batsman_id','matches_played'],ascending=[True,False]).drop_duplicates(subset=['batsman_id'])

In [201]:
test_data_prep_1st_inn.head()

,batsman_id,batsman,matches_played,strike_rate,Averge_of_batsman
2576,1,Mahendra Singh Dhoni,89,135.60,25.08
2110,3,Ravichandran Ashwin,17,85.86,5.00
2605,12,Parthiv Patel,58,114.07,18.31
2557,24,Suresh Raina,91,132.81,29.09
2606,30,Rohit Sharma,78,135.81,30.05


We can see in above data frame that each players performance in only 1st innings of the matches he played.

# Second innings

In the same way as in 1st innings the steps are followed for 2nd innings too.

In [202]:
ball_by_ball_only_2018_2ndinn.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,batsman_id,non_striker,non_striker_id,...,bowler_id,batsman_runs,extra_runs,player_dismissed,player_dismissed_id,dismissal_kind,is_super_over,season,venue_id,seasonal_match_id
88,2017_1,2,RCB,SRH,1,1,Chris Gayle,79,Mandeep Singh,373,...,119,1,0,NaN,NaN,NaN,0,2017,4,1
89,2017_1,2,RCB,SRH,1,2,Mandeep Singh,373,Chris Gayle,79,...,119,0,0,NaN,NaN,NaN,0,2017,4,1
90,2017_1,2,RCB,SRH,1,3,Mandeep Singh,373,Chris Gayle,79,...,119,0,0,NaN,NaN,NaN,0,2017,4,1
91,2017_1,2,RCB,SRH,1,4,Mandeep Singh,373,Chris Gayle,79,...,119,2,0,NaN,NaN,NaN,0,2017,4,1
92,2017_1,2,RCB,SRH,1,5,Mandeep Singh,373,Chris Gayle,79,...,119,4,0,NaN,NaN,NaN,0,2017,4,1


In [203]:
#### matchwise aggregation.

score_card_2nd_inn=ball_by_ball_only_2018_2ndinn.groupby(['match_id','seasonal_match_id','season','venue_id','batting_team','bowling_team','batsman','batsman_id'], as_index=False)['batsman_runs'].agg({'ball_palyed':'count','batsman_runs':'sum'})

In [204]:
score_card_2nd_inn.head()

,match_id,seasonal_match_id,season,venue_id,batting_team,bowling_team,batsman,batsman_id,ball_palyed,batsman_runs
0,2008_1,1,2008,7,RCB,KKR,Virat Kohli,149,5,1
1,2008_10,10,2008,9,MI,KXI,Dhawal Kulkarni,118,17,2
2,2008_10,10,2008,9,MI,KXI,Dwayne Bravo,114,22,23
3,2008_10,10,2008,9,MI,KXI,Harbhajan Singh,131,1,0
4,2008_10,10,2008,9,MI,KXI,Robin Uthappa,141,19,21


In [205]:
score_card_2nd_inn['unique_id']=(score_card_2nd_inn['match_id']).apply(str)+"_"+(score_card_2nd_inn['batsman_id']).apply(str)

In [206]:
score_card_2nd_inn=score_card_2nd_inn.sort_values(['batsman_id','season','seasonal_match_id'])

In [207]:
#### Adding up the total Runs scored for each match Cummulatively
score_card_2nd_inn['cumm_runs']=score_card_2nd_inn.groupby(['batsman_id'])['batsman_runs'].cumsum()
#### Adding the total balls faced for the each match
score_card_2nd_inn['B/F']=score_card_2nd_inn.groupby(['batsman_id'])['ball_palyed'].cumsum()

## Calculating strike rate, average and matches played

In [208]:
#### Calculating the strike rate formula: total_no_of_runs/total_balls_faced


score_card_2nd_inn['strike_rate']=np.round((score_card_2nd_inn['cumm_runs']/score_card_2nd_inn['B/F'])*100,2)



In [209]:
#### To calculate the Average the of the batsman for each match.

#### This is number of matches played.
score_card_2nd_inn['matches_played']=(score_card_2nd_inn.groupby(['batsman_id'])['B/F'].rank(ascending=True)).apply(int)


#### Average of batsman formula: Runs_scored/Matches_palyed.

score_card_2nd_inn['Averge_of_batsman']=np.round((score_card_2nd_inn['cumm_runs']/score_card_2nd_inn['matches_played']),2)




## Test set for 2018 matches (2nd innings)

In [210]:
test_data_prep_2nd_inn=score_card_2nd_inn[['batsman_id','batsman','matches_played','strike_rate','Averge_of_batsman']].sort_values(['batsman_id','matches_played'],ascending=[True,False]).drop_duplicates(subset=['batsman_id'])

In [211]:
test_data_prep_2nd_inn.head()

,batsman_id,batsman,matches_played,strike_rate,Averge_of_batsman
2498,1,Mahendra Singh Dhoni,54,128.43,24.59
2081,3,Ravichandran Ashwin,19,99.32,7.68
2494,12,Parthiv Patel,58,111.60,21.72
2408,24,Suresh Raina,66,137.87,28.68
2495,30,Rohit Sharma,75,120.35,24.84


## Shifting the average, strike rate and matches played to the next matches

Let me explain here, As before i said when ever a player is playing in one match before the start of match what we know is only his previous overall strike rate, average and matches played. So i shifted the strike rate to very next match of the player.

### 2nd Innings

In [212]:
score_card_2nd_inn.head()

,match_id,seasonal_match_id,season,venue_id,batting_team,bowling_team,batsman,batsman_id,ball_palyed,batsman_runs,unique_id,cumm_runs,B/F,strike_rate,matches_played,Averge_of_batsman
8,2008_12,12,2008,5,CSK,KKR,Mahendra Singh Dhoni,1,27,43,2008_12_1,43,27,159.26,1,43.00
36,2008_28,28,2008,2,CSK,DD,Mahendra Singh Dhoni,1,33,33,2008_28_1,76,60,126.67,2,38.00
72,2008_48,48,2008,5,CSK,RCB,Mahendra Singh Dhoni,1,8,4,2008_48_1,80,68,117.65,3,26.67
83,2008_51,51,2008,5,CSK,RR,Mahendra Singh Dhoni,1,8,12,2008_51_1,92,76,121.05,4,23.00
91,2008_55,55,2008,4,CSK,SRH,Mahendra Singh Dhoni,1,27,37,2008_55_1,129,103,125.24,5,25.80


In [213]:
### shift strike_rate
score_card_2nd_inn['s_strike_rate']=score_card_2nd_inn.groupby(['batsman_id'])['strike_rate'].shift(1)

In [214]:
### shift Average_of_batsman
score_card_2nd_inn['s_Averge_of_batsman']=score_card_2nd_inn.groupby(['batsman_id'])['Averge_of_batsman'].shift(1)

In [215]:
### Shift matches_played.

score_card_2nd_inn['s_matches_played']=score_card_2nd_inn.groupby(['batsman_id'])['matches_played'].shift(1)


### 1st Innings

In [216]:
score_card_1st_inn.head()

,match_id,seasonal_match_id,season,venue_id,batting_team,bowling_team,batsman,batsman_id,ball_palyed,batsman_runs,unique_id,cumm_runs,B/F,strike_rate,matches_played,Averge_of_batsman
26,2008_2,2,2008,9,CSK,KXI,Mahendra Singh Dhoni,1,3,2,2008_2_1,2,3,66.67,1,2.00
112,2008_8,8,2008,5,CSK,MI,Mahendra Singh Dhoni,1,17,30,2008_8_1,32,20,160.00,2,16.00
14,2008_15,15,2008,7,CSK,RCB,Mahendra Singh Dhoni,1,31,65,2008_15_1,97,51,190.20,3,32.33
29,2008_20,20,2008,5,CSK,DD,Mahendra Singh Dhoni,1,28,32,2008_20_1,129,79,163.29,4,32.25
38,2008_24,24,2008,6,CSK,RR,Mahendra Singh Dhoni,1,7,1,2008_24_1,130,86,151.16,5,26.00


In [217]:
### shift strike_rate
score_card_1st_inn['s_strike_rate']=score_card_1st_inn.groupby(['batsman_id'])['strike_rate'].shift(1)

In [218]:
### shift Average_of_batsman
score_card_1st_inn['s_Averge_of_batsman']=score_card_1st_inn.groupby(['batsman_id'])['Averge_of_batsman'].shift(1)

In [219]:
### Shift matches_played.

score_card_1st_inn['s_matches_played']=score_card_1st_inn.groupby(['batsman_id'])['matches_played'].shift(1)


### Removing very first match of each player since we dont have any use of very first match because we shifting the strike rate to next match

In [220]:
#### i dont want the very first match of the player since we dont know the average, strike rate, ball played for the previous 
#### Match since we use it for the prediction purpose in next match. 
 

score_card_1st_inn=score_card_1st_inn[~score_card_1st_inn['s_matches_played'].isnull()]

In [221]:
score_card_2nd_inn=score_card_2nd_inn[~score_card_2nd_inn['s_matches_played'].isnull()]

In [222]:
final_score_card_1st_inn=score_card_1st_inn[['unique_id','venue_id','batting_team','bowling_team','s_strike_rate','s_Averge_of_batsman','s_matches_played','batsman_runs']]

In [223]:
final_score_card_1st_inn.head()

,unique_id,venue_id,batting_team,bowling_team,s_strike_rate,s_Averge_of_batsman,s_matches_played,batsman_runs
112,2008_8_1,5,CSK,MI,66.67,2.00,1.0,30
14,2008_15_1,7,CSK,RCB,160.00,16.00,2.0,65
29,2008_20_1,5,CSK,DD,190.20,32.33,3.0,32
38,2008_24_1,6,CSK,RR,163.29,32.25,4.0,1
42,2008_26_1,5,CSK,SRH,151.16,26.00,5.0,23


In [330]:
final_score_card_2nd_inn=score_card_2nd_inn[['unique_id','venue_id','batting_team','bowling_team','s_strike_rate','s_Averge_of_batsman','s_matches_played','batsman_runs']]

In [331]:
final_score_card_2nd_inn.head()

,unique_id,venue_id,batting_team,bowling_team,s_strike_rate,s_Averge_of_batsman,s_matches_played,batsman_runs
36,2008_28_1,2,CSK,DD,159.26,43.00,1.0,33
72,2008_48_1,5,CSK,RCB,126.67,38.00,2.0,4
83,2008_51_1,5,CSK,RR,117.65,26.67,3.0,12
91,2008_55_1,4,CSK,SRH,121.05,23.00,4.0,37
115,2009_1_1,11,CSK,MI,125.24,25.80,5.0,36


### Label encoding the batting and bowling team

In [332]:
#### Label encoding the batting and bowling teams.


teams_codes={'MI':0,
        'DD':1,
        'KKR':2,
        'SRH':3,
        'CSK':4,
        'RR':5,
        'RCB':6,
        'KXI':7,
        'DC':8,
        'KTK':9,
        'RPS':10,
        'GL':11,
        'PW':12,
}

In [333]:
##### Mapping this team_codes

In [334]:
final_score_card_1st_inn=final_score_card_1st_inn.replace({'batting_team':teams_codes,'bowling_team':teams_codes})

In [335]:
final_score_card_2nd_inn=final_score_card_2nd_inn.replace({'batting_team':teams_codes,'bowling_team':teams_codes})

In [336]:
final_score_card_1st_inn.head()

,unique_id,s_strike_rate,s_Averge_of_batsman,s_matches_played,batsman_runs
112,2008_8_1_4,66.67,2.00,1.0,30
14,2008_15_1_4,160.00,16.00,2.0,65
29,2008_20_1_4,190.20,32.33,3.0,32
38,2008_24_1_4,163.29,32.25,4.0,1
42,2008_26_1_4,151.16,26.00,5.0,23


### One hot encoding the batting, bowling and venue_id

## 1st innings:

In [237]:
final_score_card_1st_inn['unique_id']=(final_score_card_1st_inn['unique_id']).apply(str)+'_'+(final_score_card_1st_inn['batting_team']).apply(str)
del final_score_card_1st_inn['batting_team']

In [252]:
##### including the 2018 1st innings data for data preparation sake.
test_data_2018_1st_innings=((player_prediction_2018[['unique_id','match_id','player_id','team_id']]).merge(matchs_2018[['match_id','team1_id','team2_id','venue_id']], on=['match_id'])).merge(test_data_prep_1st_inn, left_on='player_id', right_on='batsman_id')
test_data_2018_1st_innings['bowling_team']='NaN'

#### creating bowling team variable.

for i in range(len(test_data_2018_1st_innings)):
    if test_data_2018_1st_innings.iloc[i,3]==test_data_2018_1st_innings.iloc[i,4]:
        test_data_2018_1st_innings.iloc[i,12]=test_data_2018_1st_innings.iloc[i,5]
    else:
        test_data_2018_1st_innings.iloc[i,12]=test_data_2018_1st_innings.iloc[i,4]
        
#### Picking only important varibales.

test_data_2018_1st_innings=test_data_2018_1st_innings[['unique_id','venue_id','team_id','bowling_team','strike_rate','Averge_of_batsman','matches_played']]

### label encoding teams.
test_data_2018_1st_innings=test_data_2018_1st_innings.replace({'team_id':teams_codes,'bowling_team':teams_codes})


test_data_2018_1st_innings['unique_id']=(test_data_2018_1st_innings['unique_id']).apply(str)+'_'+(test_data_2018_1st_innings['team_id']).apply(str)

del test_data_2018_1st_innings['team_id']

#### column name changing.


test_data_2018_1st_innings.columns=['unique_id','venue_id','bowling_team','s_strike_rate','s_Averge_of_batsman','s_matches_played']
test_data_2018_1st_innings['batsman_runs']=0

test_data_2018_1st_innings.head()


,unique_id,venue_id,bowling_team,s_strike_rate,s_Averge_of_batsman,s_matches_played,batsman_runs
0,2018_1_1_4,1,0,135.6,25.08,89,0
1,2018_5_1_4,5,2,135.6,25.08,89,0
2,2018_12_1_4,8,7,135.6,25.08,89,0
3,2018_17_1_4,32,5,135.6,25.08,89,0
4,2018_20_1_4,4,3,135.6,25.08,89,0


In [257]:
##### Now concatinating with previous data it ll be easy for the preprocessing.


final_score_card_1st_inn=pa.concat([final_score_card_1st_inn,test_data_2018_1st_innings],axis=0)

In [258]:
#### one hot encoding for the batting team bowling team and venueid.


final_score_card_1st_inn['venue_id']='venue_id '+(final_score_card_1st_inn['venue_id']).apply(str)
                                        


In [259]:
venue_id_dummy=pa.get_dummies(final_score_card_1st_inn['venue_id'])

In [260]:
final_score_card_1st_inn['bowling_team']='team '+(final_score_card_1st_inn['bowling_team']).apply(str)

In [261]:
teams_dummies=pa.get_dummies(final_score_card_1st_inn['bowling_team'])

In [262]:
### remove venue_id, batting and bowling team and add the dummy values.
del final_score_card_1st_inn['bowling_team']

del final_score_card_1st_inn['venue_id']


In [263]:
final_data_1st_inn=pa.concat([final_score_card_1st_inn['unique_id'],venue_id_dummy,teams_dummies,final_score_card_1st_inn.iloc[:,1:]], axis=1)

## 2nd innings:

In [337]:
final_score_card_2nd_inn['unique_id']=(final_score_card_2nd_inn['unique_id']).apply(str)+'_'+(final_score_card_2nd_inn['batting_team']).apply(str)
del final_score_card_2nd_inn['batting_team']

In [338]:
##### including the 2018 2nd innings data for data preparation sake.
test_data_prep_2nd_inn=((player_prediction_2018[['unique_id','match_id','player_id','team_id']]).merge(matchs_2018[['match_id','team1_id','team2_id','venue_id']], on=['match_id'])).merge(test_data_prep_1st_inn, left_on='player_id', right_on='batsman_id')
test_data_prep_2nd_inn['bowling_team']='NaN'

#### creating bowling team variable.

for i in range(len(test_data_prep_2nd_inn)):
    if test_data_prep_2nd_inn.iloc[i,3]==test_data_prep_2nd_inn.iloc[i,4]:
        test_data_prep_2nd_inn.iloc[i,12]=test_data_prep_2nd_inn.iloc[i,5]
    else:
        test_data_prep_2nd_inn.iloc[i,12]=test_data_prep_2nd_inn.iloc[i,4]
        
#### Picking only important varibales.

test_data_prep_2nd_inn=test_data_prep_2nd_inn[['unique_id','venue_id','team_id','bowling_team','strike_rate','Averge_of_batsman','matches_played']]

### label encoding teams.
test_data_prep_2nd_inn=test_data_prep_2nd_inn.replace({'team_id':teams_codes,'bowling_team':teams_codes})


test_data_prep_2nd_inn['unique_id']=(test_data_prep_2nd_inn['unique_id']).apply(str)+'_'+(test_data_prep_2nd_inn['team_id']).apply(str)

del test_data_prep_2nd_inn['team_id']

#### column name changing.


test_data_prep_2nd_inn.columns=['unique_id','venue_id','bowling_team','s_strike_rate','s_Averge_of_batsman','s_matches_played']
test_data_prep_2nd_inn['batsman_runs']=0

test_data_prep_2nd_inn.head()


,unique_id,venue_id,bowling_team,s_strike_rate,s_Averge_of_batsman,s_matches_played,batsman_runs
0,2018_1_1_4,1,0,135.6,25.08,89,0
1,2018_5_1_4,5,2,135.6,25.08,89,0
2,2018_12_1_4,8,7,135.6,25.08,89,0
3,2018_17_1_4,32,5,135.6,25.08,89,0
4,2018_20_1_4,4,3,135.6,25.08,89,0


In [339]:
##### Now concatinating with previous data it ll be easy for the preprocessing.


final_score_card_2nd_inn=pa.concat([final_score_card_2nd_inn,test_data_prep_2nd_inn],axis=0)

In [340]:
#### one hot encoding for the batting team bowling team and venueid.


final_score_card_2nd_inn['venue_id']='venue_id '+(final_score_card_2nd_inn['venue_id']).apply(str)
                                        


In [342]:
venue_id_dummy=pa.get_dummies(final_score_card_2nd_inn['venue_id'])

In [343]:
final_score_card_2nd_inn['bowling_team']='team '+(final_score_card_2nd_inn['bowling_team']).apply(str)

In [344]:
teams_dummies=pa.get_dummies(final_score_card_2nd_inn['bowling_team'])

In [345]:
### remove venue_id,bowling team and add the dummy values.
del final_score_card_2nd_inn['bowling_team']

del final_score_card_2nd_inn['venue_id']



In [346]:
final_2nd_inn_data=pa.concat([final_score_card_2nd_inn['unique_id'],venue_id_dummy,teams_dummies,final_score_card_2nd_inn.iloc[:,1:]], axis=1)

# PREDICTION

## 1st innings

In [283]:
#### seperating the X and Y in the data.

#### Since the 3 variables s_strike_rate, s_average_of_batsman, s_matches_played got skewed right i used sqrt transformation
#### to this vars. Now it is alteast following the normal distribution



final_data_1st_inn['s_strike_rate']=np.sqrt(final_data_1st_inn['s_strike_rate'])
final_data_1st_inn['s_Averge_of_batsman']=np.sqrt(final_data_1st_inn['s_Averge_of_batsman'])
final_data_1st_inn['s_matches_played']=np.sqrt(final_data_1st_inn['s_matches_played'])
final_data_1st_inn['batsman_runs']=np.sqrt(final_data_1st_inn['batsman_runs'])

In [284]:
y=final_data_1st_inn.iloc[:,-1]

In [285]:
#### dependencies.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [289]:
final_data_1st_inn['season']=([(row.split('_')[0]) for row in final_data_1st_inn['unique_id']])
final_data_1st_inn['season']=(final_data_1st_inn['season']).apply(int)

In [291]:
#### for test sake let me take the 2017 data.

test_data_1st_inn=final_data_1st_inn[final_data_1st_inn['season']==2017]

In [292]:
train_data_1st_inn=final_data_1st_inn[(final_data_1st_inn['season']!=2017) & (final_data_1st_inn['season']!=2018)]

In [293]:
first_innings_2018_train_data=final_data_1st_inn[final_data_1st_inn['season']==2018]

In [294]:
trainX=train_data_1st_inn.iloc[:,1:50]
trainy=train_data_1st_inn.iloc[:,-2]

In [295]:
testX=test_data_1st_inn.iloc[:,1:50]
testy=test_data_1st_inn.iloc[:,-2]

## Lasso (Regularized regression model)

In [297]:
from sklearn.linear_model import Lasso

In [298]:
clf=Lasso(alpha=0.2)

In [299]:
clf.fit(trainX,trainy)

Lasso(alpha=0.2, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [300]:
np.round(clf.predict(testX))**2

array([ 25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  16.,  16.,  16.,
        16.,  16.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,
        25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,   9.,
         9.,  16.,  16.,  16.,  16.,  16.,  16.,  16.,  16.,  25.,  25.,
        25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,
        25.,  25.,  16.,  16.,  16.,  16.,  16.,  16.,  25.,  25.,   9.,
         9.,   9.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,
        25.,  25.,  25.,  16.,  16.,  16.,  16.,  16.,  16.,  16.,  16.,
        16.,  16.,  16.,  16.,  16.,  16.,  16.,  16.,   9.,  16.,  16.,
        16.,  16.,  16.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,
        16.,  16.,  16.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,
        25.,  25.,  25.,  16.,  16.,  16.,  16.,  16.,  25.,  25.,  16.,
        16.,  16.,  16.,  16.,  16.,  16.,  25.,  25.,  25.,  25.,  25.,
        25.,  25.,  25.,  25.,   9.,   9.,   9.,  1

In [301]:
testy**2

2473     5.0
2281     5.0
2311    28.0
2392     7.0
2399    23.0
2426    21.0
2501    31.0
2576    40.0
2337    19.0
2366     8.0
2509    25.0
2524    23.0
2605     4.0
2407    68.0
2613     5.0
2306    28.0
2437     1.0
2465     8.0
2492    77.0
2535     6.0
2557     2.0
2338     3.0
2367     5.0
2510    10.0
2525    67.0
2562    27.0
2606    24.0
2622    89.0
2271    19.0
2408     5.0
        ... 
2612    31.0
2305    14.0
2432     2.0
2459     2.0
2287     1.0
2378    18.0
2477     1.0
2379     0.0
2478    36.0
2590     0.0
2280    25.0
2308     2.0
2390    17.0
2499    39.0
2313    31.0
2394    45.0
2401    38.0
2428    37.0
2503     1.0
2578     0.0
2506     1.0
2328     6.0
2318     0.0
2442     2.0
2547     2.0
2586    22.0
2364     7.0
2604    13.0
2415     1.0
2255     4.0
Name: batsman_runs, Length: 363, dtype: float64

## Error Rate (RMSE)

In [302]:
RMSE=np.sqrt(mean_squared_error(np.round(clf.predict(testX)),testy))
print(RMSE)

2.24532440302


# 2018 Prediction using Lasso

In [305]:
trainX_2018=first_innings_2018_train_data.iloc[:,1:50]

In [310]:
predicted_score_2018=np.round(clf.predict(trainX_2018))**2

In [313]:
pa.concat([first_innings_2018_train_data['unique_id'],pa.Series(predicted_score_2018)],axis=1).head()

,unique_id,0
0,2018_1_1_4,16.0
1,2018_5_1_4,9.0
2,2018_12_1_4,9.0
3,2018_17_1_4,16.0
4,2018_20_1_4,16.0


## KNN Algorithm

In [314]:
from sklearn.neighbors import KNeighborsRegressor

clf=KNeighborsRegressor(n_neighbors=5)

In [315]:
clf.fit(trainX,trainy)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

## Error Rate (RMSE)

In [316]:
np.sqrt(mean_squared_error(np.round(clf.predict(testX)),np.ceil(testy)))

2.536238728324355

## 2018 Prediction using KNN

In [322]:
predicted_score_2018=np.round(clf.predict(trainX_2018))**2

In [323]:
pa.concat([first_innings_2018_train_data['unique_id'],pa.Series(predicted_score_2018)],axis=1).head()

,unique_id,0
0,2018_1_1_4,16.0
1,2018_5_1_4,9.0
2,2018_12_1_4,9.0
3,2018_17_1_4,16.0
4,2018_20_1_4,16.0


#######################################################################################################################

# Second innings prediction

In [348]:
final_2nd_inn_data['s_Averge_of_batsman']=np.sqrt(final_2nd_inn_data['s_Averge_of_batsman'])

In [349]:
final_2nd_inn_data['s_matches_played']=np.sqrt(final_2nd_inn_data['s_matches_played'])

In [350]:
final_2nd_inn_data['batsman_runs']=np.sqrt(final_2nd_inn_data['batsman_runs'])

In [351]:
final_2nd_inn_data['season']=([(row.split('_')[0]) for row in final_2nd_inn_data['unique_id']])
final_2nd_inn_data['season']=(final_2nd_inn_data['season']).apply(int)

In [352]:
test=final_2nd_inn_data[final_2nd_inn_data['season']==2017]
train=final_2nd_inn_data[(final_2nd_inn_data['season']!=2017) & (final_2nd_inn_data['season']!=2018)]
second_innings_2018_train_data=final_2nd_inn_data[final_2nd_inn_data['season']==2018]

In [353]:
trainX=train.iloc[:,:50]

In [354]:
trainy=train.iloc[:,-2]

In [355]:
testX=test.iloc[:,:50]
testy=test.iloc[:,-2]

In [356]:
clf=Lasso(alpha=0.2)

In [357]:
clf.fit(trainX,trainy)

Lasso(alpha=0.2, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

## Error Rate (RMSE)

In [358]:
np.sqrt(mean_squared_error(np.ceil(testy),np.round(clf.predict(testX))))

2.5478657954711514

# 2018 Prediction using Lasso

In [359]:
train_2nd_inning_2018=second_innings_2018_train_data.iloc[:,:50]

In [363]:
prediction_2018=np.round(clf.predict(train_2nd_inning_2018)**2)

In [367]:
pa.concat([train_2nd_inning_2018['unique_id'],pa.Series(prediction_2018)],axis=1).head()

,unique_id,0
0,2018_1_1_4,21.0
1,2018_5_1_4,21.0
2,2018_12_1_4,21.0
3,2018_17_1_4,21.0
4,2018_20_1_4,21.0
